# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("../output_WeatherPy/cities.csv")
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Olden,61.8364,6.8064,45.34,86,3,7.72,NO,1647342754
1,Kodino,63.7198,39.6447,28.02,74,8,8.66,RU,1647342755
2,Oranjestad,12.5240,-70.0270,76.95,83,20,11.50,AW,1647342755
3,Kyra,49.5776,111.9753,8.62,59,87,11.77,RU,1647342755
4,Grosuplje,45.9556,14.6589,53.08,58,100,8.01,SI,1647342755


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"]

In [4]:
import gmplot

# Plot Heatmap
fig = gmaps.figure(center=(20, 20), zoom_level=1.8)

# Create heat layer
HeatLayer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=300, point_radius=3.5)


# Add layer
fig.add_layer(HeatLayer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 70 and 80
narrowed_df = weather_df[(weather_df["Wind Speed"] <= 10) & (weather_df["Cloudiness"] == 0) & (weather_df["Max Temp"] >= 70) & (weather_df["Max Temp"] <= 80)]
narrowed_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
39,Lázaro Cárdenas,17.9583,-102.2000,70.56,87,0,4.03,MX,1647342766
57,Kahului,20.8947,-156.4700,74.84,79,0,8.01,US,1647342773
245,Puerto Escondido,15.8500,-97.0667,76.05,65,0,3.42,MX,1647342830
359,Arraial do Cabo,-22.9661,-42.0278,78.66,83,0,5.75,BR,1647342861
405,Springbok,-29.6643,17.8865,74.08,18,0,1.23,ZA,1647342872
509,Werda,-25.2667,23.2833,76.96,37,0,3.58,BW,1647342901


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Create a hotel_df
hotel_df = narrowed_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
39,Lázaro Cárdenas,MX,17.9583,-102.2000,
57,Kahului,US,20.8947,-156.4700,
245,Puerto Escondido,MX,15.8500,-97.0667,
359,Arraial do Cabo,BR,-22.9661,-42.0278,
405,Springbok,ZA,-29.6643,17.8865,
509,Werda,BW,-25.2667,23.2833,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "type":"lodging",
    "radius" : 5000,
    "key": g_key}

for index, row in hotel_df.iterrows():
    Lat = row['Lat']
    Lng = row["Lng"]
    city = row["City"]
    print(f"Retrieving Results for Index {index}: {city}.")
    params["location"] = f'{Lat}, {Lng}'
    response = requests.get(base_url, params=params).json()

    try:
        HotelName = response["results"][1]["name"]
        print(f"Closest hotel in {city} is {HotelName}.")
        hotel_df.loc[index, 'Hotel Name'] = HotelName
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 39: Lázaro Cárdenas.
Closest hotel in Lázaro Cárdenas is Baymont by Wyndham Lazaro Cardenas.
------------
Retrieving Results for Index 57: Kahului.
Closest hotel in Kahului is Maui Beach Hotel.
------------
Retrieving Results for Index 245: Puerto Escondido.
Closest hotel in Puerto Escondido is Hotel Suites Villasol.
------------
Retrieving Results for Index 359: Arraial do Cabo.
Closest hotel in Arraial do Cabo is Ocean View Hotel.
------------
Retrieving Results for Index 405: Springbok.
Closest hotel in Springbok is Masonic Hotel.
------------
Retrieving Results for Index 509: Werda.
Missing field/result... skipping.
------------


In [8]:
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
final_hotel_df = hotel_df.dropna()
final_hotel_df

,City,Country,Lat,Lng,Hotel Name
39,Lázaro Cárdenas,MX,17.9583,-102.2000,Baymont by Wyndham Lazaro Cardenas
57,Kahului,US,20.8947,-156.4700,Maui Beach Hotel
245,Puerto Escondido,MX,15.8500,-97.0667,Hotel Suites Villasol
359,Arraial do Cabo,BR,-22.9661,-42.0278,Ocean View Hotel
405,Springbok,ZA,-29.6643,17.8865,Masonic Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame nam

hotel_info = [info_box_template.format(**row) for index, row in final_hotel_df.iterrows()]
locations = final_hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))